In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula_middle_padding import Tabula 
import pandas as pd
import torch

In [2]:
data = pd.read_csv("Real_Datasets/real_estate_valuation_cleaned.csv")
data.columns = ["X1_house_age", "X2_distance_MRT_station", "X3_number_convenience_stores", "X4_lat", "X5_long", "X6_price"]


In [3]:
#categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
model = Tabula(llm='distilgpt2', experiment_dir = "real-estate_training", batch_size=32, epochs=128)

## In following block, it is important to indicate "conditional_col = data.columns[0]". Otherwise, the generation will use target column as conditional column, which will result in error!

In [4]:
model.fit(data, conditional_col = data.columns[0])

/home/svenlag/tabula_test/Tabula/tabula_middle_padding/tabula.py:130: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.207200
1000,0.276900
1500,0.238800


In [5]:
import torch
torch.save(model.model.state_dict(), "real-estate_training/model_128epoch_middle_padding.pt")

In [5]:
torch.save(model, "real-estate_training/model_128epoch_middle_padding_FULL.pt")

In [3]:
# Specify weights_only=False to allow full unpickling
model = torch.load(
    "real-estate_training/model_128epoch_middle_padding_FULL.pt",
    map_location=torch.device("cuda"),      # or "cpu" as needed
    weights_only=False
)
model.model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
for i in range(9,31):
    synthetic_data = model.sample(n_samples=414)
    synthetic_data.to_csv(f"real-estate_sd-sets/real-estate_128epoch_middle_padding_{i}.csv", index=False)

  0%|          | 0/414 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
499it [00:19, 25.89it/s]                         
499it [00:18, 26.38it/s]                         
500it [00:18, 26.42it/s]                         
499it [00:19, 26.24it/s]                         
498it [00:18, 26.59it/s]                         
500it [00:18, 26.71it/s]                         
500it [00:18, 26.54it/s]                         
498it [00:18, 26.59it/s]                         
500it [00:18, 26.46it/s]                         
500it [00:18, 26.82it/s]                         
499it [00:18, 26.69it/s]                         
500it [00:18, 26.87it/s]                         
499it [00:18, 26.47it/s]                         
500it [00:18, 26.79it/s]                         
500it [00:18, 26.94it/s]        

In [6]:
synthetic_data = model.sample(n_samples=414)
synthetic_data.to_csv("real-estate_128epoch_middle_padding.csv", index=False)

  0%|          | 0/414 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
489it [00:05, 92.97it/s]                          


In [7]:
from sdmetrics.reports.single_table import QualityReport

synthetic_data = synthetic_data[data.columns.to_list()]

metadata = {
    "columns": {
        "X1_house_age": {"sdtype": "numerical"},
        "X2_distance_MRT_station": {"sdtype": "numerical"},
        "X3_number_convenience_stores": {"sdtype": "numerical"},
        "X4_lat": {"sdtype": "numerical"},
        "X5_long": {"sdtype": "numerical"},
        "X6_price": {"sdtype": "numerical"}
    }
}

report = QualityReport()
report.generate(data, synthetic_data, metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 6/6 [00:00<00:00, 896.60it/s]|
Column Shapes Score: 69.12%

(2/2) Evaluating Column Pair Trends: |██████████| 15/15 [00:00<00:00, 318.79it/s]|
Column Pair Trends Score: 79.1%

Overall Score (Average): 74.11%

